In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder  
import numpy as np

In [6]:
NewYork_listings = pd.read_csv('dataset/New York City, New York_listings.csv')

# 전처리 함수

In [7]:
def list_count(string):
    return len(string.split(','))

def split_first(string):
    if type(string)==float:
        return np.nan
    
    else:
        return float(string.split(' ')[0])

    
    

def preprocessing_listings(df):
    
    # 1. remove unnecessary columns
    
    df = df.drop(['listing_url', 'scrape_id', 'name', 'host_id', 'description', 'picture_url', 
                  'host_thumbnail_url', 'neighborhood_overview', 'host_url', 'host_name', 'host_location', 'host_about', 
                  'host_picture_url', 'host_total_listings_count', 'neighbourhood', 'host_verifications', 'property_type',
                 'minimum_minimum_nights', 'minimum_maximum_nights', 'maximum_minimum_nights', 'maximum_maximum_nights',
                 'has_availability', 'calendar_last_scraped', 'first_review', 'last_review',
                 'host_listings_count', 'number_of_reviews', 'number_of_reviews_l30d', 'host_response_rate', 
                  'host_acceptance_rate', 'host_neighbourhood', 'calculated_host_listings_count_entire_homes',
                  'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms'], axis=1)
    
    # host_name은 host_id로 설명 가능해서 제거, name과 host_about, url들은 마땅히 할 수 있는 것이 없어서 제거 
    # (숙소사진, 호스트사진 포함), 
    # description, neighborhood_overview에 있는 내용은 다른 컬럼들로 설명가능하기에 삭제
    # host_total_listings_count와 host_listings_count는 아래 그림에서 보이듯이(corr) calculated_host_listings_count로 대체 가능하므로 삭제
    # 또한 calculated_host_listings_count_entire_homes, calculated_host_listings_count_private_rooms, 
    # calculated_host_listings_count_shared_rooms 또한 calculated_host_listings_count로 대체 가능하므로 삭제
    # neighbourhood는 neighbourhood_cleansed로 설명가능하므로 삭제
    # host_verification은 host_identity_verified로 설명되어있으므로 삭제
    # property_type은 room_type로 설명될 수 있으므로 삭제
    # minimum_minimum_nights, minimum_maximum_nights, maximum_minimum_nights, maximum_maximum_nights는 아래 그림에서 볼 수 있듯이
    # minimum_nights_ntm, maximum_nights_ntm과 상관관계가 높게 나오므로 삭제
    # has_availability는 availability_30, availability_60, availability_90, availability_365로 설명가능할 것이므로 삭제
    # number_of_reviews랑 number_of_reviews_l30d는 number_of_reviews_ltm으로 설명가능하므로 삭제 (밑에 corr 그림 참조)
    # host_response_rate는 주관적이고, 이거랑 host_acceptance_rate는 객관적인 host_response_time으로 대체 가능하므로 삭제 (밑에 corr 그림 참조)
    # host_neighbourhood랑 host_location은 host가 숙소근처에 있는지 여부가 중요한 변수가 될 수 있으므로 포함시키는 것이 좋을 수도 있다. 
    # 하지만 host_response_time으로 대체가능할 것이므로 삭제함
    
    
    # 2. 70%이상이 null 인경우 drop 시키기 
    # (drop 시킬 떄 문제점: 뒤에 있는 전처리 과정에서 충돌이 생길 수도 있고, host_response_time과 같은 중요한 정보를 삭제할 수 있다)
    
    for col in df.columns:
        series = df[col]
        if  series.isna().sum() / df.shape[0] > 0.7:
            df = df.drop(col, axis=1)
    
    # 3. 나머지 결측값있는 행 싸그리 제거
      
    df = df.dropna(axis=0).reset_index(drop=True)
    
    # 4. remove $ sign
    
    df['price'] = df['price'].replace('\$', '', regex=True).replace(',', '', regex=True)
    df['price'] = df['price'].astype('float')
    
    # 5. host_since를 상관관계 분석 용도로 현재 날짜-host_since를 계산해서 일수 (자연수)로 만드는 전처리 
    
    latest = pd.to_datetime(df['last_scraped'], format = '%Y-%m-%d')
    since = pd.to_datetime(df['host_since'], format = '%Y-%m-%d')

    df['host_since'] = (latest - since).dt.days
    
    df = df.drop('last_scraped', axis=1)
    
    # 6. 인코딩
    
    encoder = LabelEncoder()       
    df['host_is_superhost'] = encoder.fit_transform(df['host_is_superhost'])
    df['host_has_profile_pic'] = encoder.fit_transform(df['host_has_profile_pic'])
    df['host_identity_verified'] = encoder.fit_transform(df['host_identity_verified'])
    df['room_type'] = encoder.fit_transform(df['room_type'])
    df['instant_bookable'] = encoder.fit_transform(df['instant_bookable'])
    df['host_response_time'] = df['host_response_time'].replace({'within an hour' : 4, 'within a few hours' : 3, 'within a day' : 2, 'a few days or more': 1})
    df['neighbourhood_cleansed'] = encoder.fit_transform(df['neighbourhood_cleansed'])
    df['neighbourhood_group_cleansed'] = encoder.fit_transform(df['neighbourhood_group_cleansed'])
    
    # host_verification은 host의 본인인증정보이다 --> 리스트를 len으로 해서 얼마나 신뢰가능한 호스트인지 확인할 수 있도록 전처리
    #    (아마도 is_superhost와 상관관계가 높게나오지 않을까.)
    # 의문점: host_identity_verified는 뭐지? f가 나오는 값들은 어떤 특징들이 있을까?? --> 보류
    
    # 7. amenities를 개수로 바꿈
    
    df['amenities'] = df['amenities'].apply(list_count)
    
    # 8. bathrooms_text 에서 bathroom 개수만 뽑아냄
    
    searchfor = ['half', 'Half']
    
    tmp = df['bathrooms_text']
    
    for idx in tmp[tmp.str.contains('|'.join(searchfor))].index:
        df.loc[idx, 'bathrooms_text'] = '0.5'

    df['bathrooms_text'] = df['bathrooms_text'].apply(split_first)
    
    # 9. review 요소들을 합쳐서 하나로 만듦 (평균값)
    
    df['reviews'] = (df['review_scores_accuracy'] + df['review_scores_cleanliness'] + df['review_scores_checkin'] + df['review_scores_communication'] + df['review_scores_location'] + df['review_scores_value']) / 6
    df = df.drop(['review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value'], axis=1)
    
    return df

In [9]:
preprocessed = preprocessing_listings(NewYork_listings)
preprocessed

,id,host_since,host_response_time,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,...,availability_30,availability_60,availability_90,availability_365,number_of_reviews_ltm,review_scores_rating,instant_bookable,calculated_host_listings_count,reviews_per_month,reviews
0,3831,4324,4,0,1,1,40,1,40.68514,-73.95976,...,2,2,2,2,70,89.0,0,1,4.82,8.833333
1,5136,4265,3,0,1,0,184,1,40.66120,-73.99423,...,24,54,84,359,0,97.0,0,1,0.01,9.666667
2,5178,4236,3,0,1,0,92,2,40.76489,-73.98493,...,29,53,78,350,25,84.0,0,1,3.40,8.833333
3,5552,4245,2,0,1,1,202,2,40.73552,-74.01042,...,0,5,35,125,0,97.0,0,1,0.48,9.833333
4,5803,4234,3,1,1,1,176,1,40.66829,-73.98779,...,0,0,0,0,7,94.0,0,3,1.30,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15205,45640199,123,3,0,1,1,206,1,40.70953,-73.96352,...,5,35,65,340,2,100.0,0,1,2.00,10.000000
15206,45656925,3722,4,0,1,0,27,1,40.69944,-73.92793,...,9,39,69,69,1,100.0,0,1,1.00,10.000000
15207,45702596,664,4,0,1,1,58,3,40.75966,-73.89817,...,28,58,88,177,1,100.0,1,10,1.00,10.000000
15208,45712092,660,4,0,1,1,58,3,40.75830,-73.89812,...,23,53,83,173,1,80.0,1,10,1.00,9.666667


# outlier 제거하는 함수

In [2]:
# outlier_columns 에는 이상치가 있는 행들을 적어주어야 합니다

outlier_columns = ['price', 'reviews_per_month', 'reviews', 'calculated_host_listings_count', 'minimum_nights', 'maximum_nights']

In [4]:
def remove_outliers(df):
    for col in outlier_columns:
        top_75 = df[col].describe()['75%'] 
        top_25 = df[col].describe()['25%']
        IQR = df[col].describe()['75%'] - df[col].describe()['25%']
        df = df.loc[(df[col] >= top_25 - IQR * 1.5) & (df[col] <= top_75 + IQR * 1.5), :]
        df = df.reset_index(drop=True)
    return df

In [11]:
preprocessed = remove_outliers(preprocessed)
preprocessed

,id,host_since,host_response_time,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,...,availability_30,availability_60,availability_90,availability_365,number_of_reviews_ltm,review_scores_rating,instant_bookable,calculated_host_listings_count,reviews_per_month,reviews
0,5136,4265,3,0,1,0,184,1,40.66120,-73.99423,...,24,54,84,359,0,97.0,0,1,0.01,9.666667
1,5178,4236,3,0,1,0,92,2,40.76489,-73.98493,...,29,53,78,350,25,84.0,0,1,3.40,8.833333
2,5552,4245,2,0,1,1,202,2,40.73552,-74.01042,...,0,5,35,125,0,97.0,0,1,0.48,9.833333
3,5803,4234,3,1,1,1,176,1,40.66829,-73.98779,...,0,0,0,0,7,94.0,0,3,1.30,10.000000
4,7097,4161,4,1,1,1,78,1,40.69121,-73.97277,...,19,47,56,331,23,97.0,1,2,1.78,9.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9403,45634265,2446,4,0,1,1,42,1,40.68884,-73.99550,...,11,26,56,331,3,100.0,1,1,3.00,10.000000
9404,45639103,2560,3,0,1,1,99,2,40.87234,-73.91926,...,6,14,14,263,1,100.0,0,2,1.00,10.000000
9405,45640199,123,3,0,1,1,206,1,40.70953,-73.96352,...,5,35,65,340,2,100.0,0,1,2.00,10.000000
9406,45656925,3722,4,0,1,0,27,1,40.69944,-73.92793,...,9,39,69,69,1,100.0,0,1,1.00,10.000000
